In [68]:
#------------------------------------------------------------------------------#
# Title:        Group Project - Music Genre Classification                     #
# Course:       COMP9444 Neural Networks 2022 Term Three                       #
# Mentor:       Arun Kumar Marndi                                              #
# Session:      Thursday 16:00 - 18:00                                         #
#                                                                              #
# Team:         NNKing                                                         #
# Author/s:     Peter Huang (z5313504)                                         #
#               Fiona O’Chee (z5122503)                                        #
#               Evan Karl Lam (z5333206)                                       #
#               Theo Graftieaux (z5258743)                                     #
#               Oliver Guo (z5191682)                                          #
#                                                                              #
#------------------------------------------------------------------------------#

# Overview

### Usage Guide

No execution required; all results are already displayed for convenience. 





# Setup

In [69]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/COMP9444/COMP9444-Group-Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/COMP9444/COMP9444-Group-Project'
/content/drive/.shortcut-targets-by-id/1f2FfExUGYN2E54Q1zrEAFNotNfq9wlni/COMP9444/COMP9444-Group-Project


In [70]:
!ls # Confirm in project root directory

input  main.ipynb  output


In [71]:
import time
import os
import warnings
import IPython.display as ipd
import tqdm.notebook
import ast
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
import glob
import h5py
import keras
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D, Flatten, Reshape
import sklearn as skl
from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

In [72]:
%matplotlib inline

In [73]:
INPUT_DIR = 'input'

In [74]:
# Helper function. Conveniently load raw data into dataframes.
# (adapted from source: https://github.com/mdeff/fma)
def load(filepath):
    filename = os.path.basename(filepath)

    if 'features' in filename:
        # Use rows 0, 1 and 2 to comprise feature headings
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        columns = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for c in columns:
            tracks[c] = tracks[c].map(ast.literal_eval)

        columns = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for c in columns:
            tracks[c] = pd.to_datetime(tracks[c])

        subsets = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    'category', categories=subsets, ordered=True)
        except (ValueError, TypeError):
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=subsets, ordered=True))

        columns = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for c in columns:
            tracks[c] = tracks[c].astype('category')

        return tracks

In [ ]:
# Load data from 3 FMA files which comprise the raw dataset.
df_tracks = load(f'{INPUT_DIR}/fma_metadata/tracks.csv')
df_features = load(f'{INPUT_DIR}/fma_metadata/features.csv')
# df_echonest = load(f'{INPUT_DIR}/fma_metadata/echonest.csv')

# Check matching dimensions
# np.testing.assert_array_equal(df_features.index, df_tracks.index) 
# assert df_echonest.index.isin(df_tracks.index).all()

In [ ]:
# Filters for selecting specific row instances 
filter_small = df_tracks.index[df_tracks['set', 'subset'] <= 'small']
filter_train = df_tracks[('set', 'split')] == 'training'
filter_val = df_tracks[('set', 'split')] == 'validation'
filter_test = df_tracks[('set', 'split')] == 'test'

In [ ]:
# Create track dataframe containing only the necessary track.csv columns 
# df_tracks = df_tracks[[
#     ('set', 'split'), 
#     ('set', 'subset'),
#     ('track', 'genre_top'),
# ]]

In [ ]:
# Filter for fma_small specific music tracks only
df_tracks = df_tracks.loc[filter_small]
df_features = df_features.loc[filter_small]

In [ ]:
# Merge echonest features
# df_features = df_features.join(df_echonest, how='inner').sort_index(axis=1)

In [ ]:
# Split datasets into training, validation, testing sets.

# Note: While this is a preprocessing stage step it is 
# intentionally done early here (before exploratory data analysis) to guard 
# against Data Snooping Bias. By hiding validation and test data and only exploring 
# training data, we avoid stumbling upon interesting test set patterns which could 
# bias towards optimistic generalization error estimates.
df_tracks_train = df_tracks[filter_train]
df_tracks_validation = df_tracks[filter_val]
df_tracks_test = df_tracks[filter_test]
df_features_train = df_features.loc[filter_train]
df_features_val = df_features.loc[filter_val]
df_features_test = df_features.loc[filter_test]

In [ ]:
df_tracks.info()

# Exploratory Data Analysis

The raw dataset is provided by FMA and is composed from 3 separate files:

* tracks.csv
* features.csv
* echonest.csv

Source: fma_metadata.zip from https://github.com/mdeff/fma

### General

This sub-section aims to gain a general understanding of the raw data.

##### Tracks Dataset

In [ ]:
# Keep training set pristine; make copy solely for data exploration
df_tracks_exploration = df_tracks_train.copy()

In [ ]:
# Shows that FMA features in tracks.csv are organised (via MultiIndex) in a top 
# and sub property hierarchy, arranged as tuples
# e.g album has "sub-features" comments, date_created, ..., tags, etc
df_tracks_exploration.info()

In [ ]:
df_tracks_exploration.describe()

In [ ]:
df_tracks_exploration.shape

In [ ]:
# Show general structure, values and data types of properties for each track.
df_tracks_exploration.head()

# Interpretation: tracks.csv contains metadata on each track. Some attributes 
# could serve as features (e.g tags, artist), but these will not be used for 
# training as the project goal is to classify based on the music itself. 

# Only 'genre_top' (which is applicable for our FMA dataset size choice) will 
# be used via a later track_id join with features.csv, serving as our target 
# output variable y.

In [ ]:
# df_tracks_exploration.value_counts()

In [ ]:
# Display top level genres are 8 balanced genre classes
df_tracks_exploration['track']['genre_top'].hist(bins=50, figsize=(15, 5))

##### Features Dataset

This section explores features.csv.

In [ ]:
df_tracks_exploration.columns

In [ ]:
df_features.columns

In [ ]:
df_features.info()

In [ ]:
df_features.describe()

In [ ]:
# Display top level features
df_features.columns.levels[0]

In [ ]:
# Shows each audio feature is sub-featured in terms of a statistical measure, such as kurtosis and skew.
list(df_features.columns)

In [ ]:
df_features.shape

In [ ]:
df_features.head()

In [ ]:
df_features['tonnetz'].hist(figsize=(25,15))

In [ ]:
# attributes = ["median_house_ value", "median_income", "total_rooms", "housing_median_age"]
# scatter_matrix(features[attributes], figsize=(12, 8))

In [ ]:
df_tracks_exploration['set']

In [ ]:
df_mfcc = df_features['mfcc']
df_mfcc.head()

In [ ]:
df_mfcc.shape

In [ ]:
small = df_tracks['set', 'subset'] <= 'small'
genre_Instrumental = df_tracks['track', 'genre_top'] == 'Instrumental'
genre2_HH = df_tracks['track', 'genre_top'] == 'Hip-Hop'

print(small.shape, genre_Instrumental.shape, genre2_HH.shape)

X = df_features.loc[small & (genre_Instrumental | genre2_HH), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = df_tracks.loc[small & (genre_Instrumental | genre2_HH), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.5)
plt.show()
X.shape, y.shape

### Univariate Analysis

In [ ]:
# TODO

### Bivariate Analysis

In [ ]:
# features_corr_matrix = features.corr()
# features_corr_matrix['tonnetz'].sort_values(ascending=False)

#### Pairwise Correlations

In [ ]:
# TODO

# Data Preprocessing

### General

In [ ]:
# Random seed for repeated execution reproducibility 
RANDOM_SEED = 42

In [ ]:
genres = list(LabelEncoder().fit(df_tracks['track', 'genre_top']).classes_)
print('Genres ({}): {}'.format(len(genres), genres))

In [ ]:
def preprocess_pipeline(tracks, features, columns):
    # Encode each genre with an integer
    enc = LabelEncoder()
    genres = tracks['track', 'genre_top']

    # Split into training, validation and testing sets
    y_train = enc.fit_transform(genres[df_train])
    y_val = enc.transform(genres[df_val])
    y_test = enc.transform(genres[df_test])
    X_train = df_features.loc[df_train, columns].values
    X_val = df_features.loc[df_val, columns].values
    X_test = df_features.loc[df_test, columns].values
    X_train, y_train = shuffle(X_train, y_train, random_state=RANDOM_SEED)
    
    # Standardize features, removing mean (to 0) and scaling to unit variance.
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_val)
    scaler.transform(X_test)
    
    return y_train, y_val, y_test, X_train, X_val, X_test

In [ ]:
df_train = df_tracks_train.index
df_val = df_tracks_validation.index
df_test = df_tracks_test.index

print('Dataset split: {} training, {} validation, {} testing'.format(*map(len, [df_train, df_val, df_test])))

### Spectrograms (For CRNN Model)

In [ ]:
def get_audio_path(audio_dir, track_id):
    tid_str = '{:06d}'.format(track_id)
    return os.path.join(audio_dir, tid_str[:3], tid_str + '.mp3')

"""
Get track IDs from the mp3s in a directory.
"""
def get_trackIDs_from_dir(audio_dir):
    tids = []
    for _, dirnames, files in os.walk(audio_dir):
        if dirnames == []:
            tids.extend(int(file[:-4]) for file in files)
    return tids

In [ ]:
track_ids = get_trackIDs_from_dir(f"{INPUT_DIR}/fma_small")
len(track_ids)  # should return 8 * 1000 = 8000, 1000 songs per genre

In [ ]:
keep_cols = [
    ('set', 'split'),
    ('set', 'subset'),
    ('track', 'genre_top')
]

filepath = 'input/fma_metadata/tracks.csv'
tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])
df_all = tracks[keep_cols]
df_all = df_all[df_all[('set', 'subset')] == 'small']

df_all['track_id'] = df_all.index
df_all.head()

In [ ]:
grouped_df = df_all.groupby(('track', 'genre_top')).first().reset_index()
grouped_df.head(15)

In [ ]:
def plot_spectogram(track_id, genre):
    filename = get_audio_path(f"{INPUT_DIR}/fma_small", track_id)
    y, sr = librosa.load(filename)
    print(len(y),sr)
    spect = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=1024)
    spect = librosa.power_to_db(spect, ref=np.max)
    print(spect.shape, genre)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title(str(genre))
    plt.show()

In [ ]:
plot_spectogram(2, 'Hip-Hop') # Create spectogram for track 2

In [ ]:
# Visualize differences in spectrograms for each genre
for index, row in grouped_df.iterrows():
    track_id = int(row['track_id'])
    genre = row[('track', 'genre_top')]
    plot_spectogram(track_id, genre)

### Spectrogram for All Files (For LSTM model)

In [ ]:
'''
Read and convert file into melspectrogram, return as numpy array
'''
def read_convert_melspec(file_name,title = None):
  try:
    x,sr = librosa.load(file_name,sr = None)
  except:
    print(f"Can't read {file_name}")
    return None

  if title is None:
    title = file_name[:-4]
  spect = librosa.feature.melspectrogram(y=x, sr=sr,n_fft=2048, hop_length=1024)
  spect = librosa.power_to_db(spect, ref=np.max)
  return spect

In [ ]:
'''
Read and store all file path and store in a list
'''
def read_all_file():
  total_audio_list = []
  for i in range(156):
    if i < 10:
      string = "00"+str(i)
    elif i < 100:
      string = "0"+str(i)
    else :
      string = str(i)
    list_file = glob.glob(f"{INPUT_DIR}/fma_small/{string}/*.mp3")
    total_audio_list.extend(list_file)

In [ ]:
'''
Assign audio from all directory to each genre group, based on the csv file
'''
def assign_audio_to_genre(total_audio_list,top_genre):
  seperate_genre_dict = {}
  for i in top_genre:
    seperate_genre_dict[i] = []

  for i in total_audio_list:
    for j in seperate_genre_dict.keys():
      try:
        if tracks[('track', 'genre_top')][int(i[-10:-4].lstrip('0'))] == j:
          seperate_genre_dict[j].append(i)
          continue
      except Exception as e:
        print(f"Exception {e}and the string is {i}")
        break

In [ ]:
'''
preprocess data and store them into hdf5 file
'''
def write_genre_hdf(genre_list,genre_name):
  list_store = []
  for i in genre_list:
    converted = read_convert_melspec(i)
    if converted is None:
      continue
    temp = {"name":i,"mel_spec":converted}

    list_store.append(temp)

  file_name = "mel_spec/"+genre_name+".hdf5"

  h = h5py.File(file_name, 'w')
  group = h.create_group(genre_name)
  for item in list_store:

    group.create_dataset(item["name"].replace("/","-"),data = item["mel_spec"])
  h.close()

In [ ]:
genre_list = df_tracks['track']['genre_top']
total_audio_list = read_all_file()
separate_genre_dict = assign_audio_to_genre(total_audio_list, genre_list)

In [ ]:
'''
Convert all to json file, there will be a file for each music genre
'''
for keys, values in list(separate_genre_dict.items):
    write_genre_hdf(values, keys)

In [ ]:
'''
Reading the data out,
return a list of dictionary, each dictionary contains the name and preprocessed data
'''
def read_hdf(file_path):
    return_dict = []
    with h5py.File(file_path, 'r') as hf:
        print(list(hf.keys()))
        dataset = hf[file_path[9:-5]]
        print(len(list(dataset.keys())))
        keys = list(dataset.keys())
        for i in keys:
            return_dict.append({
                "name":i.replace("-","/"),
                "data":dataset[i][:]
            })
        return return_dict

# Model Selection, Tuning and Evaluation

### Baseline (Classical ML)

This purpose of this section is to implement a variety of machine learning models, which are then used as a baseline for performance comparison with neural network based classifiers to follow.

In [ ]:
# List of classical machine learning model candidates to use as baseline
classifiers = {
    'LR': LogisticRegression(max_iter=10000),
    'kNN': KNeighborsClassifier(n_neighbors=200),
    # 'SVCrbf': SVC(kernel='rbf', max_iter=10000),
    # 'SVCpoly1': SVC(kernel='poly', degree=1, max_iter=10000),
    # 'linSVC1': SVC(kernel="linear", max_iter=10000),
    # 'linSVC2': LinearSVC(),
    # 'DT': DecisionTreeClassifier(max_depth=5),
    # 'RF': RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    # 'AdaBoost': AdaBoostClassifier(n_estimators=10),
    # 'NB': GaussianNB(),
}

# TODO: Improve this by using subsets per: "Index(['chroma_cens', 'chroma_cqt', 'chroma_stft', 'mfcc', 'rmse',
      #  'spectral_bandwidth', 'spectral_centroid', 'spectral_contrast',
      #  'spectral_rolloff', 'tonnetz', 'zcr'],
feature_subsets = {}
for subset in df_features.columns.levels[0]:
    feature_subsets[subset] = subset
    feature_subsets.update({
        'mfcc/contrast': ['mfcc', 'spectral_contrast'],
        'mfcc/contrast/chroma': ['mfcc', 'spectral_contrast', 'chroma_cens'],
        'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
        'mfcc/contrast/chroma/centroid': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid'],
        'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
        # 'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
        # 'all': list(df_features.columns.levels[0])
    })

In [ ]:
# Make columns represent each model candidate
baseline_models = list(classifiers.keys())
# Add a column to show number of features used
baseline_models = baseline_models.insert(0, 'numFeatures')

# Setup dataframe to show scores in a matrix, with models as columns and 
# feature subsets as rows
acc_scores_matrix = pd.DataFrame(columns=baseline_models, index=feature_subsets.keys())
for fsubset_name, fsubset in tqdm.notebook.tqdm(feature_subsets.items(), desc='feature subset'):
  y_train, y_val, y_test, X_train, X_val, X_test = preprocess_pipeline(df_tracks, df_features, fsubset)

  acc_scores_matrix.loc[fsubset_name, 'numFeatures'] = X_train.shape[1]
  for clf_name, clf in classifiers.items(): 
      clf.fit(X_train, y_train)
      score = clf.score(X_test, y_test)
      acc_scores_matrix.loc[fsubset_name, clf_name] = score
      y_pred = clf.predict(X_test)
      

In [ ]:
# Helper function, highlights the top scoring for each feature subset in model-feature matrix
def format_matrix_scores(scores):
    def highlight(s):
        is_max = s == max(s[1:])
        return ['background-color: green' if v else '' for v in is_max]
    scores = scores.style.apply(highlight, axis=1)
    return scores.format('{:.1%}', subset = pd.IndexSlice[:, scores.columns[1]:])

In [ ]:
# TODO: Uncomment only when required as this runs 10+ models for 518 features
# acc_scores = run_baseline_models_vs_feature_subsets(classifiers, feature_sets)
ipd.display(format_matrix_scores(acc_scores_matrix))

### Neural Networks

#### CNN

#### LSTM

#### Transfer Learning Approach

#### GRU

#### CRNN

In [ ]:
import os
import numpy as np
from os.path import isfile

import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, Flatten, Conv2D, BatchNormalization, Lambda

from keras import backend
from keras.layers import ELU
from keras import regularizers
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

import librosa
import librosa.display

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
%matplotlib inline

In [ ]:
genres_dict = {
    'Electronic': 0, 
    'Experimental': 1, 
    'Folk': 2, 
    'Hip-Hop': 3, 
    'Instrumental': 4,
    'International': 5, 
    'Pop': 6, 
    'Rock': 7  
}

reverse_genres_dic = {v: k for k, v in genres_dict.items()}

In [ ]:
npz_file = np.load('input/shuffled_train.npz')
# print(npz_file.files)
X_train = npz_file['arr_0'] # get array from the first file 
y_train = npz_file['arr_1'] # get array from the second file
print(X_train.shape, y_train.shape)

In [ ]:
npz_valid_file = np.load('input/shuffled_valid.npz')
# print(npz_valid_file.files)
X_valid = npz_valid_file['arr_0']
y_valid = npz_valid_file['arr_1']
print(X_valid.shape, y_valid.shape)

In [ ]:
# Check by plotting a Spectogram
num = 5300
spectogram = X_train[num]
genre_index = np.argmax(y_train[num])

print(reverse_genres_dic[genre_index])

plt.figure(figsize=(10, 5))
librosa.display.specshow(spectogram.T, y_axis='mel', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Test Melspectogram')
plt.tight_layout()

##### Implementation

In [ ]:
# Hyperparameters

batch_size = 32
num_classes = 8   # 8 genres classes in total
n_features = X_train.shape[2]
n_time = X_train.shape[1]
N_LAYERS = 3
FILTER_LENGTH = 5
CONV_FILTER_COUNT = 56
BATCH_SIZE = 32
LSTM_COUNT = 96
EPOCH_COUNT = 70
NUM_HIDDEN = 64
L2_regularization = 0.001

In [ ]:
def conv_recurrent_model_build(model_input):
    print('Building model...')
    layer = model_input
    
    ### 3 1D Convolution Layers
    for i in range(N_LAYERS):
        # give name to the layers
        layer = Conv1D(
                filters=CONV_FILTER_COUNT,
                kernel_size=FILTER_LENGTH,
                kernel_regularizer=regularizers.l2(L2_regularization),  # Tried 0.001
                name='convolution_' + str(i + 1)
            )(layer)
        layer = BatchNormalization(momentum=0.9)(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling1D(2)(layer)
        layer = Dropout(0.4)(layer)
    
    ## LSTM Layer
    layer = LSTM(LSTM_COUNT, return_sequences=False)(layer)
    layer = Dropout(0.4)(layer)
    
    ## Dense Layer
    layer = Dense(NUM_HIDDEN, kernel_regularizer=regularizers.l2(L2_regularization), name='dense1')(layer)
    layer = Dropout(0.4)(layer)
    
    ## Softmax Output
    layer = Dense(num_classes)(layer)
    layer = Activation('softmax', name='output_realtime')(layer)
    model_output = layer
    model = Model(model_input, model_output)
    
    
    opt = Adam(learning_rate=0.001)
    model.compile(
            loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy']
        )
    
    print(model.summary())
    return model

In [ ]:
def train_model(x_train, y_train, x_val, y_val):
    
    n_features = x_train.shape[2]
    input_shape = (None, n_features)
    model_input = Input(input_shape, name='input')
    
    model = conv_recurrent_model_build(model_input)
    
#     tb_callback = TensorBoard(log_dir='./logs/4', histogram_freq=1, batch_size=32, write_graph=True, write_grads=False,
#                               write_images=False, embeddings_freq=0, embeddings_layer_names=None,
#                               embeddings_metadata=None)
    checkpoint_callback = ModelCheckpoint('./models/crnn/weights.best.h5', monitor='val_acc', verbose=1,
                                          save_best_only=True, mode='max')
    
    reducelr_callback = ReduceLROnPlateau(
                monitor='val_acc', factor=0.5, patience=10, min_delta=0.01,
                verbose=1
            )
    callbacks_list = [checkpoint_callback, reducelr_callback]

    # Fit the model and get training history.
    print('Training...')
    history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCH_COUNT,
                        validation_data=(x_val, y_val), verbose=1, callbacks=callbacks_list)

    return model, history

In [ ]:
model, history  = train_model(X_train, y_train, X_valid, y_valid)

In [ ]:
def show_summary_stats(history):
    # List all data in history
    print(history.history.keys())

    # Summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # Summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
show_summary_stats(history)

##### Evaluation

In [ ]:
y_true = np.argmax(y_valid, axis = 1)
y_pred = model.predict(X_valid)
y_pred = np.argmax(y_pred, axis=1)
labels = [0,1,2,3,4,5,6,7]
target_names = genres_dict.keys()

print(y_true.shape, y_pred.shape)
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
print(accuracy_score(y_true, y_pred))

# Summary, Insights and Conclusions

TODO